- Gaussian, layers 2, cells 40, epochs 30
  - test wmape 0.511
  - valid wmape 0.427
    
- Gaussian, layers 2, cells 100, epochs 30
  - test wmape 0.565
  - valid wmape 0.417

- Gaussian, layers 3, cells 60, epochs 30
  - test wmape 0.543
  - valid wmape 0.418
    
    
- Negative Binomial, layers 2, cells 40, epochs 10
  - test wmape 0.713
  - valid wmape 0.688
  
- Negative Binomial, layers 3, cells 60, epochs 50
  - test wmape 0.718
  - valid wmape 0.639

In [96]:
import warnings; warnings.filterwarnings("ignore")

In [97]:
import pandas as pd
from time import time
import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt

from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions

In [98]:
path = ".."
df = pd.read_parquet(f"{path}/data/processed/outlier_removed.parquet")

In [99]:
GENERATE_SPEED_ANGLE = True
add_lagged = True
INPUT_WIDTH = 120

train_ratio = 0.8
valid_ratio = 0.1

In [100]:
if add_lagged:
    df["production_48_lagged"] = df.groupby("rt_plant_id").production.shift(48)

SELECTED_PLANT_ID = df.groupby("rt_plant_id").production.sum().sort_values(ascending=False).index[0]
print(SELECTED_PLANT_ID)
weather_cols = [col for col in df.columns if col.startswith(("UGRD", "VGRD"))]
if add_lagged:
    df = df.set_index("forecast_dt")[["rt_plant_id", "production", "production_48_lagged", *weather_cols]]
    df = df.dropna()
else:    
    df = df.set_index("forecast_dt")[["rt_plant_id", "production", *weather_cols]]
df = df[df["rt_plant_id"] == SELECTED_PLANT_ID].drop("rt_plant_id", axis=1)

969


In [101]:
if GENERATE_SPEED_ANGLE:
    for box in ["SW", "NW", "NE", "SE"]:
        df[f"speed_{box}"] = np.sqrt(np.square(df[f"UGRD_80.m.above.ground.{box}"]) + np.square(df[f"VGRD_80.m.above.ground.{box}"]))
        df[f"angle_{box}"] = np.arctan(df[f"UGRD_80.m.above.ground.{box}"] / df[f"VGRD_80.m.above.ground.{box}"])
        
time_indices = sorted(df.index.unique())

train_indices = time_indices[:int(len(time_indices) * train_ratio)]
valid_indices = time_indices[int(len(time_indices) * train_ratio):int(len(time_indices) * (train_ratio + valid_ratio))]
test_indices = time_indices[int(len(time_indices) * (train_ratio + valid_ratio)):]

train_df = df.loc[train_indices, :]
valid_df = df.loc[valid_indices, :]
test_df = df.loc[test_indices, :]

print("Train start and end dates:\t", train_indices[0], "\t", train_indices[-1])
try:
    print("Validation start and end dates:\t", valid_indices[0], "\t", valid_indices[-1])
except:
    pass
print("Test start and end dates:\t", test_indices[0], "\t", test_indices[-1])

Train start and end dates:	 2019-01-26 03:00:00 	 2021-06-22 09:00:00
Validation start and end dates:	 2021-06-22 10:00:00 	 2021-10-10 04:00:00
Test start and end dates:	 2021-10-10 05:00:00 	 2022-01-27 23:00:00


In [102]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_df = pd.DataFrame(scaler.fit_transform(train_df), index=train_df.index, columns=train_df.columns)
try:
    valid_df = pd.DataFrame(scaler.transform(valid_df), index=valid_df.index, columns=valid_df.columns)
except:
    pass
test_df = pd.DataFrame(scaler.transform(test_df), index=test_df.index, columns=test_df.columns)

gluon_df = pd.concat([train_df, valid_df, test_df])

In [103]:
ctx = "gpu" if mx.context.num_gpus() else "cpu"
print(ctx)
PREDICTION_LENGTH = 48
CONTEXT_LENGTH = 120
freq = "1H"
EPOCHS = 30

cpu


In [53]:
feature_columns = [col for col in train_df.columns if col not in ["production"]]

def dataset_helper(df):
    return {
        "target": df.production,
        "start": df.index[0],
        "item_id": str(SELECTED_PLANT_ID),
        "feat_dynamic_real": df[feature_columns].T
    }

train_ds = ListDataset([dataset_helper(df=gluon_df[gluon_df.index < valid_indices[0]])], freq=freq)
valid_ds = ListDataset([dataset_helper(df=gluon_df[gluon_df.index < test_indices[0]])], freq=freq)
valid_ds = ListDataset([dataset_helper(df=gluon_df[gluon_df.index < valid_indices[date_shift]]) for date_shift in [(i+1)*24 for i in range(len(valid_indices) // 24)]], freq=freq)
test_ds = ListDataset([dataset_helper(df=gluon_df[gluon_df.index < test_indices[date_shift]]) for date_shift in [(i+1)*24 for i in range(len(test_indices) // 24)]], freq=freq)

In [83]:
from gluonts.mx.distribution import NegativeBinomialOutput, GaussianOutput, StudentTOutput

mx.random.seed(7)
np.random.seed(7)
deepar_estimator = DeepAREstimator(
    freq=freq, 
    num_layers=3,
    num_cells=60,
    prediction_length=PREDICTION_LENGTH, 
    context_length=CONTEXT_LENGTH, 
    use_feat_dynamic_real=True, 
    trainer=Trainer(epochs=50, ctx=ctx), # learning_rate=1e-3, hybridize=False, num_batches_per_epoch=20
    # distr_output=GaussianOutput()
    distr_output=NegativeBinomialOutput()
    # distr_output=StudentTOutput()
)
predictor = deepar_estimator.train(training_data=train_ds, validation_data=valid_ds, num_workers=None)

100%|█████████| 50/50 [00:17<00:00,  2.78it/s, epoch=1/50, avg_epoch_loss=0.532]
4it [00:00,  4.25it/s, epoch=1/50, validation_avg_epoch_loss=0.614]
100%|█████████| 50/50 [00:21<00:00,  2.32it/s, epoch=2/50, avg_epoch_loss=0.856]
4it [00:01,  3.89it/s, epoch=2/50, validation_avg_epoch_loss=0.727]
100%|██████████| 50/50 [00:17<00:00,  2.88it/s, epoch=3/50, avg_epoch_loss=0.97]
4it [00:01,  3.92it/s, epoch=3/50, validation_avg_epoch_loss=0.726]
100%|█████████████| 50/50 [00:16<00:00,  2.99it/s, epoch=4/50, avg_epoch_loss=1]
4it [00:01,  3.24it/s, epoch=4/50, validation_avg_epoch_loss=0.726]
100%|█████████| 50/50 [00:20<00:00,  2.49it/s, epoch=5/50, avg_epoch_loss=0.967]
4it [00:00,  4.10it/s, epoch=5/50, validation_avg_epoch_loss=0.724]
100%|█████████| 50/50 [00:19<00:00,  2.52it/s, epoch=6/50, avg_epoch_loss=0.945]
4it [00:01,  3.90it/s, epoch=6/50, validation_avg_epoch_loss=0.725]
100%|██████████| 50/50 [00:18<00:00,  2.72it/s, epoch=7/50, avg_epoch_loss=1.03]
4it [00:00,  4.20it/s, ep

In [92]:
valid_forecast_it, _ = make_evaluation_predictions(valid_ds, predictor=predictor, num_samples=100)
valid_predictions = []
for forecast_entry in valid_forecast_it:
    valid_predictions.append(forecast_entry.mean_ts[-24:])
valid_predictions = pd.concat(valid_predictions)

test_forecast_it, _ = make_evaluation_predictions(test_ds, predictor=predictor, num_samples=100)
test_predictions = []
for forecast_entry in test_forecast_it:
    test_predictions.append(forecast_entry.mean_ts[-24:])
test_predictions = pd.concat(test_predictions)

In [85]:
def calculate_wmape(preds, actuals):
    return np.sum(np.abs(preds-actuals)) / np.sum(np.abs(actuals))

print("Validation WMAPE:", calculate_wmape(valid_predictions, valid_df.production[:len(valid_predictions)]))
print("Test WMAPE:", calculate_wmape(test_predictions, test_df.production[:len(test_predictions)]))

Validation WMAPE: 0.6398227251072479
Test WMAPE: 0.7185554521139558


In [93]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])

test_forecast_it, _ = make_evaluation_predictions(test_ds, predictor=predictor, num_samples=100)
agg_metrics, item_metrics = evaluator(iter(list(_)), iter(list(test_forecast_it)), num_series=len(test_ds))

Running evaluation: 100%|████████████████████| 109/109 [00:00<00:00, 199.82it/s]


In [94]:
agg_metrics

{'MSE': 0.08951946858328054,
 'abs_error': 1904.2221754789352,
 'abs_target_sum': 1960.5813292264938,
 'abs_target_mean': 0.37472884732922274,
 'seasonal_error': 0.2174525484705995,
 'MASE': 1.6716798560555823,
 'MAPE': 0.9868940186105688,
 'sMAPE': 1.960508875213949,
 'MSIS': 9.877702817334402,
 'QuantileLoss[0.1]': 392.11626755751206,
 'Coverage[0.1]': 0.0,
 'QuantileLoss[0.5]': 1904.222181775025,
 'Coverage[0.5]': 0.019877675840978593,
 'QuantileLoss[0.9]': 919.4803531188516,
 'Coverage[0.9]': 0.9332951070336392,
 'RMSE': 0.2991980424121798,
 'NRMSE': 0.7984387765837403,
 'ND': 0.9712538557276816,
 'wQuantileLoss[0.1]': 0.20000000087331918,
 'wQuantileLoss[0.5]': 0.9712538589390198,
 'wQuantileLoss[0.9]': 0.468983530247946,
 'mean_absolute_QuantileLoss': 1071.9396008171295,
 'mean_wQuantileLoss': 0.5467457966867616,
 'MAE_Coverage': 0.2044724770642202,
 'OWA': nan}

In [95]:
item_metrics

,item_id,MSE,abs_error,abs_target_sum,abs_target_mean,seasonal_error,MASE,MAPE,sMAPE,MSIS,QuantileLoss[0.1],Coverage[0.1],QuantileLoss[0.5],Coverage[0.5],QuantileLoss[0.9],Coverage[0.9]
0,969.0,0.119947,3.147887,3.147887,0.065581,0.213573,0.307065,1.000000,2.000000,11.315398,0.629577,0.0,3.147887,0.000000,11.612676,0.958333
1,969.0,0.021589,6.267606,6.267606,0.130575,0.213397,0.611888,1.000000,2.000000,8.200670,1.253521,0.0,6.267606,0.000000,8.647887,0.916667
2,969.0,0.047059,12.443663,12.443663,0.259243,0.213252,1.215665,1.000000,2.000000,5.846477,2.488732,0.0,12.443662,0.000000,14.209859,0.479167
3,969.0,0.052704,10.119719,10.119719,0.210827,0.213165,0.989035,1.000000,2.000000,5.570808,2.023944,0.0,10.119718,0.000000,13.384507,0.625000
4,969.0,0.030937,3.455070,3.455070,0.071981,0.213158,0.337687,1.000000,2.000000,7.330240,0.691014,0.0,3.455070,0.000000,8.628704,0.958333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,969.0,0.036567,18.225351,19.950705,0.415640,0.222244,1.708461,0.949866,1.860981,10.967683,3.990141,0.0,18.225352,0.083333,10.009859,1.000000
105,969.0,0.024137,16.841549,16.841549,0.350866,0.222559,1.576504,1.000000,2.000000,9.548017,3.368310,0.0,16.841549,0.000000,7.431690,1.000000
106,969.0,0.055977,22.919014,22.919014,0.477479,0.222642,2.144602,1.000000,2.000000,10.386607,4.583803,0.0,22.919014,0.000000,6.016197,1.000000
107,969.0,0.096451,31.580986,31.580986,0.657937,0.222554,2.956303,1.000000,2.000000,11.326835,6.316197,0.0,31.580986,0.000000,5.683803,0.979167


In [89]:
# pd.concat([valid_predictions, valid_df.production[:len(valid_predictions)]], axis=1).iloc[:200].plot(figsize=(20,12))

In [90]:
# pd.concat([test_predictions, test_df.production[:len(test_predictions)]], axis=1).iloc[:500].plot(figsize=(20,12))